In [31]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver import ChromeOptions
import time
import pandas as pd
import datetime
import os
import numpy as np

In [32]:
def append_to_dict (dict, key, var):
    if key in dict.keys():
        dict[key].append(var)
    else:
        dict[key] = [var]
    return dict

def equalise_dict(dict):
    max_len = 0
    for key in dict:
        if len(dict[key]) > max_len:
            max_len = len(dict[key])
    for key in dict:
        if len(dict[key]) < max_len:
            for _ in range(max_len - len(dict[key])):
                dict[key].append(list(dict[key])[0])
    return dict

def lower_case (df):
    for c in df.columns:
        df[str(c).lower()] = df[c].apply(lambda x: str(x).lower())
        if str(c).lower() != c:
            df.drop(c, axis=1, inplace=True)
    return df

In [7]:
def scrape_jockey (driver, jockey_name):

    jockey_csv_path = os.path.join("data","1_scrape",'jockeys',f"{jockey_name}.csv")

    jockey_dict = {}

    for row in driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[2]/table/tbody/tr'):
        if 'age' in row.find_element(By.TAG_NAME, 'td').text.lower():
            jockey_dict = append_to_dict(jockey_dict, 'jockey_age', row.find_element(By.TAG_NAME, 'td').text.lower().replace('age:','').strip())
        if'career wins' in row.find_element(By.TAG_NAME, 'td').text.lower():
            jockey_dict = append_to_dict(jockey_dict, 'career_wins', row.find_element(By.TAG_NAME, 'td').text.lower().strip())

    jockey_dict = append_to_dict(jockey_dict, 'nationality', driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]/td[2]').text.lower().replace(':','').strip())

    season = driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/thead/tr/td').text.lower().split(' ')[0]
    for row in driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr'):
        num_cols = len(row.find_elements(By.TAG_NAME, 'td'))
        for i in range(num_cols/2):
            var_name = '_'.join([str(season), row.find_elements(By.TAG_NAME, 'td')[i].text.lower()])
            if 'nationality' in var_name:
                continue
            if var_name == '':
                continue
            var_value = row.find_elements(By.TAG_NAME, 'td')[i].text.lower().replace(':','').strip()
            jockey_dict = append_to_dict(jockey_dict, var_name, var_value)

    if driver.find_elements(By.XPATH, '//*[@id="seasonTextId"]'):
        driver.find_element(By.XPATH, '//*[@id="seasonTextId"]').click()
        WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]/td[2]').is_displayed())
        season = driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/thead/tr/td').text.lower().split(' ')[0]
        for row in driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr'):
            num_cols = len(row.find_elements(By.TAG_NAME, 'td'))
            for i in range(num_cols/2):
                var_name = '_'.join([str(season), row.find_elements(By.TAG_NAME, 'td')[i].text.lower()])
                if 'nationality' in var_name:
                    continue
                if var_name == '':
                    continue
                var_value = row.find_elements(By.TAG_NAME, 'td')[i].text.lower().replace(':','').strip()
                jockey_dict = append_to_dict(jockey_dict, var_name, var_value)

    # click on performance
    driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[1]/div/ul[1]/li[2]/a').click()
    WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]').is_displayed())
    # click on previous season
    if driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[2]/table/tbody/tr[8]/td/a'):
        driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[2]/table/tbody/tr[8]/td/a').click()
        WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]').is_displayed())
    for row in driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr'):
        cells = row.find_elements(By.TAG_NAME, 'td')
        if len(cells) == 7:
            var_name_1 = cells[0].text.lower()
            var_name_1 += '_' + cells[1].text.lower()
        elif len(cells) == 6:
            var_name_1 = cells[0].text.lower()
        for jockey_perf_idx in range(4):
            var_name = var_name_1 + '_' + cells[-5].text.lower() + '_' + driver.find_element(By.XPATH, f'//*[@id="innerContent"]/div[2]/div[3]/table/thead/tr/td[{jockey_perf_idx + 4}]').text.lower()
            var_value = cells[-jockey_perf_idx - 1].text
            jockey_dict = append_to_dict(jockey_dict, var_name, var_value)

    # # click on riding records
    # driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[1]/div/ul[1]/li[3]/a').click()
    # WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]').is_displayed())
    # time.sleep(0.5)
    # for row in driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr'):
    #     cells = row.find_elements(By.TAG_NAME, 'td')
    #     if len(cells) == 1:
    #         date_loc = cells[0].find_elements(By.TAG_NAME, 'span')[0].text
    #     else:
    #         for cell_idx, cell in enumerate(cells):
    #             var_name = ' '.join(driver.find_element(By.XPATH, f'//*[@id="innerContent"]/div[2]/div[3]/table/thead/tr/td[{cell_idx + 1}]').text.splitlines())
    #             if cell_idx == 0:
    #                 var_value = date_loc + ' ' + cell.text
    #             else:
    #                 var_value = cell.text
    #             jockey_dict = append_to_dict(jockey_dict, var_name, var_value)

    # # click on previous season
    # driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[2]/table/tbody/tr[8]/td/a').click()
    # time.sleep(1)
    # if len(driver.find_elements(By.ID, 'errorContainer')) > 0:
    #     driver.back()
    # else:
    #     done = False
    #     while not done:
    #         for row in driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr'):
    #             cells = row.find_elements(By.TAG_NAME, 'td')
    #             if len(cells) == 1:
    #                 date_loc = cells[0].find_elements(By.TAG_NAME, 'span')[0].text
    #             else:
    #                 for cell_idx, cell in enumerate(cells):
    #                     var_name = ' '.join(driver.find_element(By.XPATH, f'//*[@id="innerContent"]/div[2]/div[3]/table/thead/tr/td[{cell_idx + 1}]').text.splitlines())
    #                     if cell_idx == 0:
    #                         var_value = date_loc + ' ' + cell.text
    #                     else:
    #                         var_value = cell.text
    #                     jockey_dict = append_to_dict(jockey_dict, var_name, var_value)
    #         if len(driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/p[1]/span[2]/a')) > 0:
    #             driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/p[1]/span[2]/a').click()
    #             time.sleep(1)
    #         else:
    #             done = True

    jockey_dict = equalise_dict(jockey_dict)
    jockey_df = pd.DataFrame(jockey_dict)
    jockey_df = lower_case(jockey_df)
    jockey_df.to_csv(jockey_csv_path)
    print(jockey_csv_path, 'saved')

In [33]:
# scrape jockeys
driver = Chrome()
# driver = Chrome()

driver.get("https://racing.hkjc.com/racing/information/English/Racing/LocalResults.aspx")
WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.CSS_SELECTOR, "option").is_displayed())
dates = driver.find_elements(By.CSS_SELECTOR, "option")
num_of_dates = len(dates)

for date_idx in range(num_of_dates):
    date_dropdown = driver.find_element(by = By.XPATH, value = "//select[@id = 'selectId']")
    date_dropdown.click()
    dates = driver.find_elements(By.CSS_SELECTOR, "option")
    date = dates[date_idx+1]
    race_date = datetime.datetime.strptime(date.text, '%d/%m/%Y')
    today = datetime.date.today().strftime("%d/%m/%Y")

    # if race_date >= datetime.datetime.today():
    if race_date > datetime.datetime(year=2023, month=5, day=13):
        continue

    date.click()
    submit_button = driver.find_element(By.ID, "submitBtn")
    submit_button.click()
    current_url = driver.current_url
    if not "local" in current_url.lower():
        driver.back()
        continue
    WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/table/tbody/tr[1]/td').is_displayed())

    race_buttons_indices = []

    for cell_idx, cell in enumerate(driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/table/tbody/tr[1]/td')):
        if len(cell.find_elements(By.TAG_NAME, 'img')) > 0:
            race_buttons_indices.append(cell_idx + 1)

    for race_button_idx in race_buttons_indices[:-1]:
        driver.find_element(By.XPATH, f'//*[@id="innerContent"]/div[2]/div[2]/table/tbody/tr[1]/td[{race_button_idx}]').click()
        WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[4]').is_displayed())
        if driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[4]/table'):
            race_no = int(driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[4]/table/thead/tr/td[1]').text[5:7].replace(" ",""))
        else:
            if 'abandoned' in driver.find_element(By.XPATH,'//*[@id="innerContent"]/div[2]/div[4]').text:
                continue

        # horse table
        num_horses = len(driver.find_elements(By.XPATH, "/html/body/div[1]/div[3]/div[2]/div[2]/div[2]/div[5]/table/tbody/tr"))

        for horse_idx in range(num_horses):
            jockey_name = driver.find_element(By.XPATH, f'//*[@id="innerContent"]/div[2]/div[5]/table/tbody/tr[{horse_idx+1}]/td[4]').text.lower()
            
            # save jockey data separately
            if '-' in jockey_name:
                continue

            jockey_csv_path = os.path.join("data","1_scrape",'jockeys',f"{jockey_name}.csv")
            if os.path.exists(jockey_csv_path):
                print(jockey_csv_path, 'already exists')
                continue
        
            race_url = driver.current_url

            # click on jockey
            driver.find_element(By.XPATH, f'//*[@id="innerContent"]/div[2]/div[5]/table/tbody/tr[{horse_idx+1}]/td[4]/a').click()
            WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[2]/table/tbody/tr').is_displayed())
            time.sleep(0.5)
            
            # scrape_jockey(driver, jockey_name)

            jockey_dict = {}

            if 'age' in driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[2]/table/tbody/tr[2]/td').text.lower():
                jockey_dict = append_to_dict(jockey_dict, 'jockey_age', driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[2]/table/tbody/tr[2]/td').text.lower().replace('age:','').strip())
            else:
                jockey_dict = append_to_dict(jockey_dict, 'jockey_age', np.nan)

            #     if'career wins' in row.find_element(By.TAG_NAME, 'td').text.lower():
            #         jockey_dict = append_to_dict(jockey_dict, 'career_wins', row.find_element(By.TAG_NAME, 'td').text.lower().split(':')[-1].strip())

            # jockey_dict = append_to_dict(jockey_dict, 'nationality', driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]/td[2]').text.lower().replace(':','').strip())

            # season = driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/thead/tr/td').text.lower().split(' ')[0]
            # for row in driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr'):
            #     num_cols = len(row.find_elements(By.TAG_NAME, 'td'))
            #     for i in range(int(num_cols/2)):
            #         var_name = '_'.join([str(season), row.find_elements(By.TAG_NAME, 'td')[i].text.lower()])
            #         if 'nationality' in var_name:
            #             continue
            #         if var_name == '':
            #             continue
            #         var_value = row.find_elements(By.TAG_NAME, 'td')[i].text.lower().replace(':','').strip()
            #         jockey_dict = append_to_dict(jockey_dict, var_name, var_value)

            # if driver.find_elements(By.XPATH, '//*[@id="seasonTextId"]'):
            #     driver.find_element(By.XPATH, '//*[@id="seasonTextId"]').click()
            #     WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]/td[2]').is_displayed())
            #     season = driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/thead/tr/td').text.lower().split(' ')[0]
            #     for row in driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr'):
            #         num_cols = len(row.find_elements(By.TAG_NAME, 'td'))
            #         for i in range(int(num_cols/2)):
            #             var_name = '_'.join([str(season), row.find_elements(By.TAG_NAME, 'td')[i].text.lower()])
            #             if 'nationality' in var_name:
            #                 continue
            #             if var_name == '':
            #                 continue
            #             var_value = row.find_elements(By.TAG_NAME, 'td')[i].text.lower().replace(':','').strip()
            #             jockey_dict = append_to_dict(jockey_dict, var_name, var_value)

            # click on performance
            driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[1]/div/ul[1]/li[2]/a').click()
            WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]').is_displayed())
            # click on previous season
            if driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[2]/table/tbody/tr[8]/td/a'):
                driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[2]/table/tbody/tr[8]/td/a').click()
                # WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]').is_displayed())
                try:
                    driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]')
                except:
                    driver.back()
                WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]').is_displayed())
            
            for row in driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[2]/table/tbody/tr')[1:]:
                for col_idx in range(int(len(row.find_elements(By.TAG_NAME, 'td'))/2)):
                    var_name = row.find_elements(By.TAG_NAME, 'td')[col_idx * 2].text.lower()
                    if var_name == '':
                        continue
                    var_value = row.find_elements(By.TAG_NAME, 'td')[col_idx * 2 + 1].text.lower().replace(':','').strip()
                    jockey_dict = append_to_dict(jockey_dict, var_name, var_value)

            for row in driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr'):
                cells = row.find_elements(By.TAG_NAME, 'td')
                if len(cells) == 7:
                    var_name_1 = cells[0].text.lower()
                    var_name_1 += '_' + cells[1].text.lower()
                elif len(cells) == 6:
                    var_name_1 = cells[0].text.lower()
                for jockey_perf_idx in range(4):
                    var_name = var_name_1 + '_' + cells[-5].text.lower() + '_' + driver.find_element(By.XPATH, f'//*[@id="innerContent"]/div[2]/div[3]/table/thead/tr/td[{jockey_perf_idx + 4}]').text.lower()
                    var_value = cells[-jockey_perf_idx - 1].text
                    jockey_dict = append_to_dict(jockey_dict, var_name, var_value)

            jockey_dict = equalise_dict(jockey_dict)
            jockey_df = pd.DataFrame(jockey_dict)
            jockey_df = lower_case(jockey_df)
            jockey_df.to_csv(jockey_csv_path)
            print(jockey_csv_path, 'saved')

            driver.get(race_url)
            WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[4]').is_displayed())

data/1_scrape/jockeys/z purton.csv already exists
data/1_scrape/jockeys/k c leung.csv already exists
data/1_scrape/jockeys/l ferraris.csv already exists
data/1_scrape/jockeys/y l chung.csv already exists
data/1_scrape/jockeys/m chadwick.csv already exists
data/1_scrape/jockeys/m f poon.csv already exists
data/1_scrape/jockeys/r maia.csv already exists
data/1_scrape/jockeys/h bowman.csv already exists
data/1_scrape/jockeys/h bentley.csv already exists
data/1_scrape/jockeys/c l cheung.csv already exists
data/1_scrape/jockeys/c l chau.csv already exists
data/1_scrape/jockeys/k teetan.csv already exists
data/1_scrape/jockeys/c y ho.csv already exists
data/1_scrape/jockeys/a badel.csv already exists
data/1_scrape/jockeys/m chadwick.csv already exists
data/1_scrape/jockeys/h bowman.csv already exists
data/1_scrape/jockeys/c y ho.csv already exists
data/1_scrape/jockeys/l ferraris.csv already exists
data/1_scrape/jockeys/k h chan.csv already exists
data/1_scrape/jockeys/z purton.csv already e

IndexError: list index out of range

In [29]:
for file_name in [f for f in os.listdir(os.path.join('data','1_scrape','jockeys')) if not f.startswith('.')]:
    df = pd.read_csv(os.path.join('data','1_scrape','jockeys', file_name), index_col=0)
    if 'jockey_age' not in df.columns:
        os.remove(os.path.join('data','1_scrape','jockeys', file_name))
        continue
    if len(df['jockey_age'].unique()) > 1:
        os.remove(os.path.join('data','1_scrape','jockeys', file_name))
        continue
    try:
        int(df['jockey_age'].unique()[0])
    except:
        os.remove(os.path.join('data','1_scrape','jockeys', file_name))
        